The following code sleeps for a random amount of time and then prints "Done!" after it wakes

In [10]:
import time
import random

def sleep_random():
    time.sleep(random.random())
    return "Done!"

sleep_random()

'Done!'

We can add a line of code called decorators that can add behaviors of the function.

The following @stopwatch is a decorator that measures the amount of time that function took.

Note: there is an error where my python library does not recognize the decorator

In [19]:
try:
    @stopwatch
    def sleep_random():
        time.sleep(random.random())
        return "Done!"
except NameError:
    pass

sleep_random()

'Done!'

We can actually write our own decorators

First lets fo over some properties of functions:

Functions can recieve functions as input

In [20]:
def add(a, b):
    return a + b

def sub(a, b):
    return a - b

def apply(func, a, b):
    return func(a,b)

apply(add, 1, 2), apply(sub, 1, 2)

(3, -1)

Functions can also return functions as outputs

In [21]:
def power(n):
    def func(number):
        return number**n
    return func

pow2 = power(2)
pow3 = power(3)

pow2(3), pow3(3)


(9, 27)

Now we can create our stopwatch decorator with the two ideas we mentioned above

In [22]:
def stopwatch(f):
    def func():
        tic = time.time()
        result = f()
        print(f"this function took: {time.time() - tic}")
        return result
    return func

def sleep_random():
    time.sleep(random.random())
    return "Done!"

timed_sleep = stopwatch(sleep_random)

In [23]:
sleep_random()

'Done!'

So now we can use functions to add behaviors to other functions

In [25]:
timed_sleep()

this function took: 0.8692836761474609


'Done!'

To make our function work better and be more flexable, we can add arguements and key word arguements to the stopwatch function

In [30]:
def stopwatch(f):
    def func(*args, **kwargs):
        tic = time.time()
        result = f(*args, **kwargs)
        print(f"this function took: {time.time() - tic}")
        return result
    return func

def sleep_random(s=1):
    time.sleep(s + random.random()/100)
    return "Done!"

timed_sleep = stopwatch(sleep_random)

In [31]:
sleep_random(s=2)

'Done!'

In [32]:
timed_sleep(s=2)

this function took: 2.009556770324707


'Done!'

If we want to use the timed_sleep function more than the actual sleep_random function. Now we can introduce the decorator syntax '@'

In [33]:
def stopwatch(f):
    def func(*args, **kwargs):
        tic = time.time()
        result = f(*args, **kwargs)
        print(f"this function took: {time.time() - tic}")
        return result
    return func

@stopwatch
def sleep_random(s=1):
    time.sleep(s + random.random()/100)
    return "Done!"

Now the function runs without the need to call the timed_function 

In [34]:
sleep_random(1), sleep_random(2)

this function took: 1.008159875869751
this function took: 2.001110315322876


('Done!', 'Done!')

An important step in making these functions and their formating is adding docstrings that we can add to the help()

help() actually calls the stopwatch function so the docstring we have in the sleep_random function wont be returned; but we can fix that by adding wraps()

In [39]:
from functools import wraps

def stopwatch(f):
    @wraps(f)
    def func(*args, **kwargs):
        tic = time.time()
        result = f(*args, **kwargs)
        print(f"this function took: {time.time() - tic}")
        return result
    return func

@stopwatch
def sleep_random(s):
    """This function sleeps for at least 's' seconds."""
    time.sleep(s + random.random()/100)
    return "Done!"

In [40]:
help(sleep_random)

Help on function sleep_random in module __main__:

sleep_random(s)
    This function sleeps for at least 's' seconds.



The following shows we can stack the decorators on top of each other. The order does matter

In [45]:
def print_call1(f):
    @wraps(f)
    def func(*args, **kwargs):
        print(f"print-call 2 args: {args}")
        result = f(*args, **kwargs)
        return result
    return func

def print_call2(f):
    @wraps(f)
    def func(*args, **kwargs):
        print(f"print-call 2 args: {args}")
        result = f(*args, **kwargs)
        return result
    return func

@print_call1
@print_call2
def sleep_random(s):
    """This function sleeps for at least 's' seconds."""
    time.sleep(s + random.random()/100)
    return "Done!"

sleep_random(1.5)

print-call 2 args: (1.5,)
print-call 2 args: (1.5,)


'Done!'

Lets make it so that we can customize the decorators further

We can add customizing features to the decorator by wrapping the the stopwatch decorator where we can now add extra if statements so that the user can pass the,

In [48]:
def loggg(show_time=True, show_name=True):
    def stopwatch(f):
        @wraps(f)
        def func(*args, **kwargs):
            tic = time.time()
            result = f(*args, **kwargs)
            log_text = "call"
            if show_name:
                log_text = f"{log_text} name: {f.__name__}"
            if show_time:
                log_text = f"{log_text} time: {time.time() - tic}"
            print(log_text)
            return result
        return func
    return stopwatch

@loggg(show_time=True, show_name=True)
def sleep_random(s):
    """This function sleeps for at least 's' seconds."""
    time.sleep(s + random.random()/100)
    return "Done!"

sleep_random(1)

call name: sleep_random time: 1.006127119064331


'Done!'

While we did allow the decorator to accomplish more with the extra parameters. we now need to add the paranthesis to make sure we can run it properly

In [49]:
def loggg(show_time=True, show_name=True):
    def stopwatch(f):
        @wraps(f)
        def func(*args, **kwargs):
            tic = time.time()
            result = f(*args, **kwargs)
            log_text = "call"
            if show_name:
                log_text = f"{log_text} name: {f.__name__}"
            if show_time:
                log_text = f"{log_text} time: {time.time() - tic}"
            print(log_text)
            return result
        return func
    return stopwatch

@loggg
def sleep_random(s):
    """This function sleeps for at least 's' seconds."""
    time.sleep(s + random.random()/100)
    return "Done!"

sleep_random(1)

<function __main__.loggg.<locals>.stopwatch.<locals>.func>

Lets noe modify the code so that we can fix this

In [50]:
def loggg(func_in=None, *, show_time=True, show_name=True):
    def stopwatch(f):
        @wraps(f)
        def func(*args, **kwargs):
            tic = time.time()
            result = f(*args, **kwargs)
            log_text = "call"
            if show_name:
                log_text = f"{log_text} name: {f.__name__}"
            if show_time:
                log_text = f"{log_text} time: {time.time() - tic}"
            print(log_text)
            return result
        return func

    if func_in is None:
        return stopwatch
    else:
        return stopwatch(func_in)

@loggg
def sleep_random(s):
    """This function sleeps for at least 's' seconds."""
    time.sleep(s + random.random()/100)
    return "Done!"

sleep_random(1)

call name: sleep_random time: 1.0031464099884033


'Done!'

The star in the logg function creates a seperating line where everything to the right of it needs to be given as a key word arg while the left can be normal args. Now with the added func_in and if-else, we do not need the paranthesis

Decorators can be very powerful, sepically when paring with the pandas library, to log the effects of pipeline steps

In [51]:
import datetime as dt

def log_step(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        tic = dt.datetime.now()
        result = func(*args, **kwargs)
        time_taken = str(dt.datetime.now() - tic)
        print(f"just ran step {func.__name__} shape = {result.shape} took {time_taken}s")
        return result
    return wrapper

In [62]:
import pandas as pd

df = pd.read_csv('data/bigmac.csv')

@log_step
def start_pipline(dataf):
    return dataf.copy()

@log_step
def set_dtypes(dataf):
    return (dataf
            .assign(data=lambda d: pd.to_datetime(d['date']))
            .sort_values(['currency_code', 'date']))
    
@log_step
def remove_outliers(dataf, min_row_country=32):
    countries = (dataf.groupby('currency_code')
                 .agg(n=('name', 'count'))
                 .loc[lambda d: d['n'] >= min_row_country]
                 .index)
    return (dataf
            .loc[lambda d: d['currency_code'].isin(countries)])


In [63]:
df_new = (df
          .pipe(start_pipline)
          .pipe(set_dtypes)
          .pipe(remove_outliers, min_row_country=20))

just ran step start_pipline shape = (1330, 6) took 0:00:00s
just ran step set_dtypes shape = (1330, 7) took 0:00:00.003002s
just ran step remove_outliers shape = (1248, 7) took 0:00:00.003000s


Another useful decorator is the @retry, where it allows to to retry the code if it fails

Can be used if the code requires internet, and the connection sometimes fails

In [66]:
from retry import retry

import logging
logging.basicConfig()

@retry(ValueError, tries=5, delay=0.5)
def randomly_fails(p=0.5):
    if random.random() < p:
        raise ValueError("OH NO!!")
    return "FINISHED!!"

randomly_fails()

'FINISHED!!'